In [19]:
import os

from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv
import pandas as pd
from switch import prompt_templates

In [20]:
load_dotenv()

True

In [21]:
all_pages = []

DIRPATH = "./data/pdfs/"

for filename in os.listdir(DIRPATH):
    filepath = os.path.join(DIRPATH, filename)
    loader = PyPDFLoader(filepath)
    pages = loader.load_and_split()
    all_pages.extend(pages)

In [22]:
embeddings = OpenAIEmbeddings()
vectors = FAISS.from_documents(all_pages, embeddings)

In [23]:
def search_similar(query: str):
    similars = vectors.similarity_search(query, k=3)
    
    return [s.page_content for s in similars]

In [24]:
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")

prompt = PromptTemplate(
    input_variables=["contents", "query"],
    template=str(prompt_templates("Default"))+            
        "{contents} {query}"
)

chain = LLMChain(llm=model, prompt=prompt)

In [25]:
def reply(query: str):
    similars = search_similar(query)
    response = chain.run(contents=similars, query=query)

    return response

In [26]:
reply("What do you think of these papers?")

'Based on the given information, it seems that the papers are discussing the Prize Collecting Traveling Salesman Problem (PCTSP) and proposing different approaches to solve it. The first paper focuses on mathematical formulations and compares the performance of different exact approaches. The second paper discusses heuristics and compares their performance with a baseline heuristic. Both papers provide results and conclusions based on their experiments. Without further context or specific criteria, it is difficult to form a definitive opinion on the papers.'

In [27]:
recom_prompt = prompt_templates("Ranking")
reply(recom_prompt)

'1. Title: "A Branch-and-Cut Algorithm for the Prize-Collecting Traveling Salesman Problem"\n   Content: This paper presents a branch-and-cut algorithm for solving the Prize-Collecting Traveling Salesman Problem (PCTSP). It describes the mathematical formulation and proposes two new MIP-based heuristics. The paper also provides numerical results to evaluate the proposed approaches.\n   Relevance: This paper is highly relevant as it directly addresses the PCTSP and proposes new algorithms for solving it.\n\n2. Title: "A Lagrangian Heuristic for the Prize-Collecting Traveling Salesman Problem"\n   Content: This paper presents a Lagrangian heuristic for obtaining an upper bound for the Prize-Collecting Traveling Salesman Problem (PCTSP). It describes the relaxation of the minimum prize constraint and the application of the sub-gradient method. The paper also discusses the similarities between the PCTSP and the Quota TSP.\n   Relevance: This paper is relevant as it provides a heuristic app